In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, r2_score

# 1) Build modeling table (NO LOG): route_id, target, lags

In [2]:
def build_model_table_no_log(
    df: pd.DataFrame,
    target_col: str = "fare_real",
    horizon: int = 1,                 # quarters ahead
    lags: tuple = (1, 4),
    lag_feature_cols: list | None = None,
):
    d = df.copy()

    # Required columns check
    required_cols = ["Year", "quarter", "citymarketid_1", "citymarketid_2", "nsmiles", target_col]
    missing = [c for c in required_cols if c not in d.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    # route_id (directionless)
    a = d["citymarketid_1"].astype(int)
    b = d["citymarketid_2"].astype(int)
    d["route_from"] = np.minimum(a, b)
    d["route_to"]   = np.maximum(a, b)
    d["route_id"]   = d["route_from"].astype(str) + "_" + d["route_to"].astype(str)

    # time index
    d["qtr_index"] = d["Year"].astype(int) * 4 + (d["quarter"].astype(int) - 1)
    d = d.sort_values(["route_id", "qtr_index"]).reset_index(drop=True)

    # Target in levels (NO LOG)
    d["y"] = pd.to_numeric(d[target_col], errors="coerce")
    d["y_target"] = d.groupby("route_id")["y"].shift(-horizon)

    # Lags of target
    for L in lags:
        d[f"y_lag{L}"] = d.groupby("route_id")["y"].shift(L)

    # Lag other features (forecast-safe)
    if lag_feature_cols is None:
        lag_feature_cols = []
    for c in lag_feature_cols:
        if c not in d.columns:
            continue
        for L in lags:
            d[f"{c}_lag{L}"] = d.groupby("route_id")[c].shift(L)

    # Feature list
    feature_cols = ["route_id", "quarter", "nsmiles"] + [f"y_lag{L}" for L in lags]
    for c in lag_feature_cols:
        for L in lags:
            colname = f"{c}_lag{L}"
            if colname in d.columns:
                feature_cols.append(colname)

    # Drop rows missing target or required lag history
    required_for_model = ["y_target"] + [f"y_lag{L}" for L in lags]
    model_df = d.dropna(subset=required_for_model).copy()

    # Make categorical
    model_df["route_id"] = model_df["route_id"].astype("category")

    return model_df, feature_cols

# 2) Time-series cross validation with LightGBM + R^2

In [3]:
def lgbm_timeseries_cv(
    model_df: pd.DataFrame,
    feature_cols: list,
    *,
    target_col: str = "y_target",
    time_col: str = "qtr_index",
    n_splits: int = 5,
    weight_col: str | None = "passengers",   # set None to disable
    params: dict | None = None,
    num_boost_round: int = 5000,
    early_stopping_rounds: int = 200,
    verbose_eval: int = 200,
    seed: int = 42,
):
    if params is None:
        params = {
            "objective": "regression",
            "metric": "mae",
            "learning_rate": 0.05,
            "num_leaves": 63,
            "min_data_in_leaf": 50,
            "feature_fraction": 0.8,
            "bagging_fraction": 0.8,
            "bagging_freq": 1,
            "seed": seed,
        }

    dfv = model_df.sort_values(time_col).reset_index(drop=True).copy()
    dfv["route_id"] = dfv["route_id"].astype("category")

    X = dfv[feature_cols]
    y = dfv[target_col].to_numpy()

    tscv = TimeSeriesSplit(n_splits=n_splits)

    fold_rows = []
    models = []

    all_y = []
    all_pred = []

    for fold, (tr_idx, va_idx) in enumerate(tscv.split(X), 1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y[tr_idx], y[va_idx]

        # Optional weights
        w_tr = None
        w_va = None
        if weight_col is not None and weight_col in dfv.columns:
            w_tr = dfv.iloc[tr_idx][weight_col].fillna(0).clip(lower=0)
            w_va = dfv.iloc[va_idx][weight_col].fillna(0).clip(lower=0)

        dtrain = lgb.Dataset(
            X_tr, label=y_tr, weight=w_tr,
            categorical_feature=["route_id"], free_raw_data=False
        )
        dvalid = lgb.Dataset(
            X_va, label=y_va, weight=w_va,
            categorical_feature=["route_id"], reference=dtrain, free_raw_data=False
        )

        model = lgb.train(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            valid_sets=[dtrain, dvalid],
            valid_names=["train", "valid"],
            callbacks=[
                lgb.early_stopping(early_stopping_rounds),
                lgb.log_evaluation(verbose_eval)
            ],
        )

        pred = model.predict(X_va, num_iteration=model.best_iteration)

        mae = float(mean_absolute_error(y_va, pred))
        rmse = float(np.sqrt(np.mean((y_va - pred) ** 2)))
        r2 = float(r2_score(y_va, pred))

        fold_rows.append({
            "fold": fold,
            "train_rows": int(len(tr_idx)),
            "valid_rows": int(len(va_idx)),
            "mae": mae,
            "rmse": rmse,
            "r2": r2,
            "best_iteration": int(model.best_iteration),
            "valid_time_min": int(dfv.loc[va_idx, time_col].min()),
            "valid_time_max": int(dfv.loc[va_idx, time_col].max()),
        })
        models.append(model)

        all_y.append(y_va)
        all_pred.append(pred)

    results_df = pd.DataFrame(fold_rows)

    all_y = np.concatenate(all_y)
    all_pred = np.concatenate(all_pred)

    summary = {
        "mae_mean": float(results_df["mae"].mean()),
        "mae_std": float(results_df["mae"].std(ddof=1)),
        "rmse_mean": float(results_df["rmse"].mean()),
        "rmse_std": float(results_df["rmse"].std(ddof=1)),
        "r2_mean": float(results_df["r2"].mean()),
        "r2_std": float(results_df["r2"].std(ddof=1)),
        "overall_cv_r2": float(r2_score(all_y, all_pred)),
    }

    return results_df, summary, models

# 3) RUN: build table -> CV -> print results

## Add cpi_adj

In [4]:
df = pd.read_excel("airline_ticket_dataset.xlsx")
cpi = pd.read_excel("CPI US.xlsx", sheet_name="Monthly")

def add_cpi(airline_ticket: pd.DataFrame, cpi:pd.DataFrame):
    airline_ticket["fare_per_miles"] = airline_ticket["fare"]/df["nsmiles"]
    
    cpi["Year"] = cpi["observation_date"].dt.year
    cpi["month"] = cpi["observation_date"].dt.month
    cpi["quarter"] = (cpi["month"] - 1) // 3 + 1

    cpi_q = (
        cpi.groupby(["Year", "quarter"], as_index=False)
           .agg(cpi_q=("CPIAUCSL", "mean"),
                months_in_q=("CPIAUCSL", "count"))
           .sort_values(["Year", "quarter"])
    )

    cpi_q["cpi_adj"] = cpi_q["cpi_q"]/284.905667 * 100
    cpi_q.drop([14, 15, 16], axis=0, inplace=True)
    
    airline_ticket = airline_ticket.merge(cpi_q[["Year", "quarter", "cpi_adj"]], on=["Year", "quarter"], how="right")
    nom_price = ["fare", "fare_lg", "fare_low"]

    for x in nom_price:
        airline_ticket[x + "_real"] = airline_ticket[x] * (100 / airline_ticket["cpi_adj"])
        
    return airline_ticket, cpi_q

df, cpi_q = add_cpi(df, cpi)
df.to_excel("cpi_added.xlsx", index=False)

import warnings
warnings.filterwarnings("ignore")

## Run

In [5]:
# Choose which columns you want lagged (forecast-safe)
lag_feature_cols = [
    "passengers",
    "large_ms", "lf_ms",
    "fare_lg_real", "fare_low_real",
    "TotalPerLFMkts_city1", "TotalPerPrem_city1",
    "TotalPerLFMkts_city2", "TotalPerPrem_city2",
]

model_df, feature_cols = build_model_table_no_log(
    df,
    target_col="fare_real",
    horizon=1,              # next quarter
    lags=(1, 4),
    lag_feature_cols=lag_feature_cols
)

cv_results, cv_summary, cv_models = lgbm_timeseries_cv(
    model_df=model_df,
    feature_cols=feature_cols,
    target_col="y_target",
    time_col="qtr_index",
    n_splits=5,
    weight_col="passengers",   # set None if you don't want weighting
    # params=...,              # optionally pass your own params dict
)

print("FEATURES USED:")
print(feature_cols)

print("\nCV RESULTS (per fold):")
print(cv_results)

print("\nCV SUMMARY:")
print(cv_summary)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000468 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4203
[LightGBM] [Info] Number of data points in the train set: 1386, number of used features: 23
[LightGBM] [Info] Start training from score 210.743327
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[59]	train's l1: 9.56837	valid's l1: 14.6928
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000650 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5844
[LightGBM] [Info] Number of data points in 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Number of data points in the train set: 5532, number of used features: 23
[LightGBM] [Info] Start training from score 209.466400
Training until validation scores don't improve for 200 rounds
[200]	train's l1: 4.77297	valid's l1: 14.9264
[400]	train's l1: 2.91439	valid's l1: 14.7882
[600]	train's l1: 1.97456	valid's l1: 14.7301
[800]	train's l1: 1.40402	valid's l1: 14.724
[1000]	train's l1: 1.03401	valid's l1: 14.7447
Early stopping, best iteration is:
[829]	train's l1: 1.34097	valid's l1: 14.7202
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000602 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6274
[LightGBM] [Info] Number of data points in the train 

# 4) Recursive Prediction, Hybrid Blend

In [6]:
import numpy as np
import pandas as pd
import lightgbm as lgb

def make_route_id_and_time(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    a = d["citymarketid_1"].astype(int)
    b = d["citymarketid_2"].astype(int)
    d["route_from"] = np.minimum(a, b)
    d["route_to"]   = np.maximum(a, b)
    d["route_id"]   = d["route_from"].astype(str) + "_" + d["route_to"].astype(str)
    d["qtr_index"]  = d["Year"].astype(int) * 4 + (d["quarter"].astype(int) - 1)
    return d

def build_recursive_safe_table(df: pd.DataFrame, target_col="fare_real", horizon=1, lags=(1,4)):
    d = make_route_id_and_time(df).sort_values(["route_id","qtr_index"]).reset_index(drop=True)
    d["y"] = pd.to_numeric(d[target_col], errors="coerce")
    d["y_target"] = d.groupby("route_id")["y"].shift(-horizon)
    for L in lags:
        d[f"y_lag{L}"] = d.groupby("route_id")["y"].shift(L)

    feature_cols = ["route_id", "quarter", "nsmiles"] + [f"y_lag{L}" for L in lags]
    model_df = d.dropna(subset=["y_target"] + [f"y_lag{L}" for L in lags]).copy()
    model_df["route_id"] = model_df["route_id"].astype("category")
    return model_df, feature_cols

def train_lgbm(model_df, feature_cols, weight_col="passengers", seed=42, num_boost_round=2000):
    X = model_df[feature_cols]
    y = model_df["y_target"].to_numpy()
    w = None
    if weight_col is not None and weight_col in model_df.columns:
        w = model_df[weight_col].fillna(0).clip(lower=0)

    dtrain = lgb.Dataset(
        X, label=y, weight=w,
        categorical_feature=["route_id"], free_raw_data=False
    )
    params = {
        "objective": "regression",
        "metric": "mae",
        "learning_rate": 0.05,
        "num_leaves": 63,
        "min_data_in_leaf": 50,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "bagging_freq": 1,
        "seed": seed,
    }
    return lgb.train(params, dtrain, num_boost_round=num_boost_round)

# Train horizon=1 recursive-safe model
h1_df, h1_features = build_recursive_safe_table(df, target_col="fare_real", horizon=1, lags=(1,4))
model_h1 = train_lgbm(h1_df, h1_features, weight_col="passengers")

# Train horizon=2 direct model (same feature set, just horizon=2)
h2_df, h2_features = build_recursive_safe_table(df, target_col="fare_real", horizon=2, lags=(1,4))
model_h2 = train_lgbm(h2_df, h2_features, weight_col="passengers")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000456 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1744
[LightGBM] [Info] Number of data points in the train set: 8296, number of used features: 5
[LightGBM] [Info] Start training from score 210.441481
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000244 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1717


In [7]:
def advance_quarter(year: int, quarter: int, steps: int = 1):
    q_index = year * 4 + (quarter - 1) + steps
    new_year = q_index // 4
    new_quarter = (q_index % 4) + 1
    return int(new_year), int(new_quarter)

def get_last4_history(d_route: pd.DataFrame, target_col: str, last_qtr_index: int):
    # assumes d_route sorted by qtr_index
    hist = d_route[d_route["qtr_index"] <= last_qtr_index].dropna(subset=[target_col])
    if len(hist) < 4:
        raise ValueError(f"Need at least 4 historical quarters up to qtr_index={last_qtr_index}.")
    return hist[target_col].tail(4).to_list()

def hybrid_recursive_forecast(
    model_h1,
    model_h2,
    df: pd.DataFrame,
    *,
    cm1: int,
    cm2: int,
    last_known_year: int,
    last_known_quarter: int,
    years_ahead: int = 5,
    blend_w: float = 0.6,                 # weight on direct h=2 prediction (0.5–0.8 typical)
    damping_alpha: float | None = 0.85,   # smooth step-to-step jumps
    clip_global_quantiles: tuple[float, float] | None = (0.01, 0.99),
    target_col: str = "fare_real",
):
    d = make_route_id_and_time(df).sort_values(["route_id","qtr_index"]).reset_index(drop=True)
    route_id = f"{min(int(cm1), int(cm2))}_{max(int(cm1), int(cm2))}"

    # route slice
    d_route = d[d["route_id"] == route_id].copy()
    if d_route.empty:
        raise ValueError(f"Route {route_id} not found in data.")

    # locate last known
    last_qi = int(last_known_year) * 4 + (int(last_known_quarter) - 1)
    row_last = d_route[d_route["qtr_index"] == last_qi]
    if row_last.empty:
        raise ValueError(f"No row for route {route_id} at {last_known_year} Q{last_known_quarter}.")

    nsmiles = float(row_last["nsmiles"].iloc[0])

    # global clipping bounds
    lo, hi = None, None
    if clip_global_quantiles is not None:
        qlo, qhi = clip_global_quantiles
        hist_all = pd.to_numeric(d[target_col], errors="coerce").dropna()
        if len(hist_all) > 0:
            lo = float(hist_all.quantile(qlo))
            hi = float(hist_all.quantile(qhi))

    # rolling last4 values for y_lag4..y_lag1 window: [t-3,t-2,t-1,t]
    d_route[target_col] = pd.to_numeric(d_route[target_col], errors="coerce")
    last4 = get_last4_history(d_route, target_col, last_qi)
    prev = float(last4[-1])

    total_quarters = int(years_ahead) * 4
    out = []

    # We forecast in 2-quarter chunks: (t+1, t+2), then advance anchor by 2
    steps_done = 0
    anchor_year, anchor_quarter = last_known_year, last_known_quarter

    while steps_done < total_quarters:
        # ---- predict t+1 using h=1 model ----
        y_lag1 = float(last4[-1])
        y_lag4 = float(last4[0])
        y1_year, y1_qtr = advance_quarter(anchor_year, anchor_quarter, 1)

        X1 = pd.DataFrame([{
            "route_id": route_id,
            "quarter": int(y1_qtr),
            "nsmiles": nsmiles,
            "y_lag1": y_lag1,
            "y_lag4": y_lag4
        }])
        X1["route_id"] = X1["route_id"].astype("category")
        pred1 = float(model_h1.predict(X1, num_iteration=getattr(model_h1, "best_iteration", None))[0])

        if damping_alpha is not None:
            pred1 = damping_alpha * pred1 + (1 - damping_alpha) * prev
        if lo is not None and hi is not None:
            pred1 = float(np.clip(pred1, lo, hi))

        out.append({"route_id": route_id, "Year": y1_year, "quarter": y1_qtr, "pred_fare_real": pred1})
        steps_done += 1
        if steps_done >= total_quarters:
            break

        # ---- predict t+2 two ways ----
        y2_year, y2_qtr = advance_quarter(anchor_year, anchor_quarter, 2)

        # (A) direct t->t+2 using h=2 model (from anchor state)
        X2_direct = pd.DataFrame([{
            "route_id": route_id,
            "quarter": int(y2_qtr),
            "nsmiles": nsmiles,
            "y_lag1": float(last4[-1]),   # anchor t
            "y_lag4": float(last4[0])     # anchor t-3
        }])
        X2_direct["route_id"] = X2_direct["route_id"].astype("category")
        pred2_direct = float(model_h2.predict(X2_direct, num_iteration=getattr(model_h2, "best_iteration", None))[0])

        # (B) recursive t+1 -> t+2 using h=1 model, where lag1 becomes pred1
        X2_rec = pd.DataFrame([{
            "route_id": route_id,
            "quarter": int(y2_qtr),
            "nsmiles": nsmiles,
            "y_lag1": float(pred1),       # use predicted t+1
            "y_lag4": float(last4[1])     # this approximates the correct lag4 for t+1 anchor shift
        }])
        X2_rec["route_id"] = X2_rec["route_id"].astype("category")
        pred2_rec = float(model_h1.predict(X2_rec, num_iteration=getattr(model_h1, "best_iteration", None))[0])

        # Blend
        pred2 = float(blend_w * pred2_direct + (1 - blend_w) * pred2_rec)

        # Damping + clipping
        if damping_alpha is not None:
            pred2 = damping_alpha * pred2 + (1 - damping_alpha) * pred1
        if lo is not None and hi is not None:
            pred2 = float(np.clip(pred2, lo, hi))

        out.append({"route_id": route_id, "Year": y2_year, "quarter": y2_qtr, "pred_fare_real": pred2})
        steps_done += 1

        # ---- advance anchor by 2 quarters ----
        # Update last4 to reflect adding pred1, pred2
        last4 = last4[2:] + [pred1, pred2]   # drop two oldest, append two newest
        prev = pred2
        anchor_year, anchor_quarter = y2_year, y2_qtr

    return pd.DataFrame(out)

## Example Usage

In [8]:
forecast_hybrid = hybrid_recursive_forecast(
    model_h1=model_h1,
    model_h2=model_h2,
    df=df,
    cm1=32467, cm2=30852, # can be adjusted based on citymarketid_1 and citymarketid_2
    last_known_year=2025,
    last_known_quarter=2,
    years_ahead=10,
    blend_w=0.9,          # lean more on direct t+2 (usually reduces drift)
    damping_alpha=0.85,
    clip_global_quantiles=(0.01, 0.99)
)

forecast_hybrid

,route_id,Year,quarter,pred_fare_real
0,30852_32467,2025,3,179.259600
1,30852_32467,2025,4,172.766105
2,30852_32467,2026,1,175.889820
3,30852_32467,2026,2,179.659335
4,30852_32467,2026,3,179.045196
5,30852_32467,2026,4,176.331413
6,30852_32467,2027,1,177.329005
7,30852_32467,2027,2,179.673814
8,30852_32467,2027,3,179.047367
9,30852_32467,2027,4,176.481768


Used blend_w = 0.9 because it was the best blend_w after comparing several other values.